In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_horizVert_wavelet_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,2835.94,-0.02,0.04,-0.00,0.00,-0.00,0.0
3,-0.02,960.90,-0.01,0.01,0.00,0.00,-0.0
4,0.04,-0.01,272.21,0.00,-0.00,0.00,-0.0
5,-0.00,0.01,0.00,62.49,-0.00,-0.00,0.0
6,0.00,0.00,-0.00,-0.00,11.94,0.00,0.0
7,-0.00,0.00,0.00,-0.00,0.00,2.07,-0.0
8,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.3


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,-0.00001,0.00005,-0.00000,0.00001,-0.00006,0.00001
3,-0.00001,1.00000,-0.00001,0.00003,0.00000,0.00008,-0.00005
4,0.00005,-0.00001,1.00000,0.00003,-0.00001,0.00003,-0.00001
5,-0.00000,0.00003,0.00003,1.00000,-0.00001,-0.00001,0.00003
6,0.00001,0.00000,-0.00001,-0.00001,1.00000,0.00002,0.00005
7,-0.00006,0.00008,0.00003,-0.00001,0.00002,1.00000,-0.00001
8,0.00001,-0.00005,-0.00001,0.00003,0.00005,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.06678165096636726

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.15335353e+06 1.28263865e+05 9.87845372e+03 4.88396852e+02
 1.58966499e+01 3.60443453e-01 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.997865,-0.063293,-0.015687,-0.003569,-0.000681,-0.000120,-0.000017
1,0.062147,0.996028,-0.062380,-0.012997,-0.002473,-0.000425,-0.000062
2,0.019260,0.060353,0.996050,-0.061216,-0.010959,-0.001890,-0.000271
3,0.005476,0.016201,0.059361,0.995811,-0.066501,-0.010966,-0.001562
4,0.001384,0.004087,0.014447,0.064429,0.993751,-0.089067,-0.012150
5,0.000356,0.001043,0.003692,0.016104,0.085512,0.985891,-0.142939
6,0.000103,0.000304,0.001072,0.004671,0.024443,0.141284,0.989656


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00213492, 0.00397229, 0.00395007, 0.00418942, 0.00624897,
       0.0141086 , 0.01034438])